In [5]:
from dolfin import *
import mshr
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML 
import time

In [11]:
def stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f, folder_name):     
    dt = T / num_steps # time step size
    t = dt
    mesh_f_s = []
    
    
    
    #input mesh choose
    if mesh_f == 0:
        mesh = Mesh('stenosis_f0.6.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_facet_region.xml')
        mesh_f_s = 'R' #regular
    else:
        mesh = Mesh('stenosis_f0.6_fine.xml')
        boundary = MeshFunction('size_t', mesh,'stenosis_f0.6_fine_facet_region.xml')
        mesh_f_s = 'F' #fine
    h = CellDiameter(mesh)
    
    
    
    # Build function space
    P2 = VectorElement("P", mesh.ufl_cell(), element_n)
    P1 = FiniteElement("P", mesh.ufl_cell(), 1)
    TH = P2 * P1
    W = FunctionSpace(mesh, TH)
    u_par = Constant((0,0))
    
    
    
    #boundary
    ds = Measure('ds', domain=mesh, subdomain_data=boundary)
    n_v = FacetNormal(mesh)
    boundary_conditions = {1: {'Dirichlet': u_par},   #inflow
                           #2: {'Dirichlet': Constant(0)},   #outflow
                           #3: {'Dirichlet': Constant(0)},   #symmetry
                           4: {'Dirichlet': Constant((0.0, 0.0))}}   #no-slip
        # Collect Dirichlet conditions
    bc = DirichletBC(W.sub(0), boundary_conditions[1]['Dirichlet'], boundary, 1)
    
    

    # Define variational problem
    (u, p) = TrialFunctions(W)
    (v, q) = TestFunctions(W)
    u0 = project(Constant((0, 0)), VectorFunctionSpace(mesh, "P", 4))
    p0 = project(Constant(0), FunctionSpace(mesh, "P", 4))
    
    
    
    #f constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))

    
    
    #file work
    folder =  folder_name + '/cn_' +str(conv_f)+ '-bc_' +str(bcflow_f)+ '-te_' +str(temam_f)+ '-su_' +str(supg_f)+ '-co_' +str(combi_f) + '_'+mesh_f_s+ '-dt_' + str(dt) + '-e_' + str(epsilon) + '-th_' + str(theta)  
    u_file = XDMFFile(folder + '/u.xdmf')
    p_file = XDMFFile(folder + '/p.xdmf')
    u_file.parameters['rewrite_function_mesh'] = False
    p_file.parameters['rewrite_function_mesh'] = False
    u_file.parameters["flush_output"] = True
    p_file.parameters["flush_output"] = True
    
    
    
    # Time-stepping
    proceent = 0
    t_arr = np.zeros(num_steps)
    
    
    
    #constant
    f = Constant((0, 0))
    f0 = Constant((0, 0))
    
    
    
    x, y = sym.symbols('x[0], x[1]')  
    u_par = (1.5 *Re *mu/2/rho/R)*(1 - (y/R)**2)
    u_D = sym.simplify(u_par)
    u_par_c = Expression((sym.printing.ccode(u_D), 0), degree=5)
    
    for n in range(num_steps):
        
        
        #progress
        if progress_f == 1:
            if int(n/(num_steps-1)*100) >= proceent:
                print("% =", int(n/(num_steps-1) * 100), ', time mplsd =', int((time.time()-t_start)))
                proceent+=proceents
        t_arr[n] = t
        
        
        #calculate u_bulk
        u_par = u_par_c*np.sin(np.pi*t/T)
        u_par = project(u_par, VectorFunctionSpace(mesh, "P", 4))
        bc_4 = DirichletBC(W.sub(0), u_par, boundary, 4)
        
        
        
        #boundary calculation
        bcs = []
        bcs.append(bc)
        bcs.append(bc_4)
        bc_3 = - inner(p0*n_v, v)*ds
        
        
        
        #Form parts calculation 
        a1 = (mu*inner(grad(u0), grad(v)) + div(v)*p0 + mu*q*div(u0))*dx(domain = mesh)
        L1 = inner(f0, v)*dx(domain = mesh)
        a2 = (mu*inner(grad(u), grad(v)) + div(v)*p + mu*q*div(u))*dx(domain = mesh) 
        L2 = inner(f, v)*dx(domain = mesh)
        
        
        
        
        #correction terms
        correction_t_r = temam_t_r = epsilon_t_r = conv_t_r = bcflow_t_r = supg_t_r = combi_t_r = temam_t_l = epsilon_t_l = conv_t_l = bcflow_t_l = supg_t_l = combi_t_l = 0
        
        
        if bcflow_f == 1:
            ala = inner(u0,n_v)
            #bcflow_t = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u - (1-theta)*u0), v) * dx(domain = mesh) 
            bcflow_t_l = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner((theta*u), v) * ds
            bcflow_t_r = bcflow_f * rho/4 * abs(ala - abs(ala)) * inner(((1-theta)*u0), v) * ds
        else:
            bcflow_t_l = bcflow_t_r = 0
        
        
        
        if temam_f == 1:
            #temam_t = temam_f * rho/2 * inner(dot(grad(u0),(theta*u - (1-theta)*u0)), v)*dx(domain = mesh) 
            temam_t_l = temam_f * rho/2 * inner(dot(grad(u0),(theta*u)), v)*dx(domain = mesh)
            temam_t_r = temam_f * rho/2 * inner(dot(grad(u0),((1-theta)*u0)), v)*dx(domain = mesh)                                   
        else:
            temam_f_t_l = temam_f_t_r = 0
        
        
        
        if epsilon == 1:
            #epsilon_t =  epsilon *h*h/mu*inner(grad(q), grad((theta*p - (1-theta)*p0)))*dx(domain = mesh)
            epsilon_t_l =  epsilon *h*h/mu*inner(grad(q), grad(theta*p))*dx(domain = mesh) 
            epsilon_t_r =  epsilon *h*h/mu*inner(grad(q), (1-theta) * grad(p0))*dx(domain = mesh)
        else:
            epsilon_t_l = epsilon_t_r = 0
        
        
        
        if conv_f == 1:                              
            #conv_t =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), v) * dx(domain = mesh) 
            conv_t_l =  conv_f * rho*inner(dot(u0, nabla_grad((theta*u))), v) * dx(domain = mesh)
            conv_t_r =  conv_f * rho*inner(dot(u0, (1-theta)*nabla_grad((u0))), v) * dx(domain = mesh)                                    
        else:
            conv_t_l = conv_t_r = 0
        
        
        
        if supg_f == 1:                              
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #supg_t =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u - (1-theta)*u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)
            supg_t_l =  supg_t_ * rho * inner(dot(u0, nabla_grad((theta*u))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                     
            supg_t_r =  supg_t_ * rho * inner(dot(u0, (1-theta)*nabla_grad((u0))), rho * dot(u0, nabla_grad(v))) * dx(domain = mesh)                                                                    
        else:
            supg_t_l = supg_t_r = 0
        
        
        
        if combi_f == 1:
            supg_t_ =  supg_f * (4/dt/dt + 4*dot(u0,u0)/h/h + (12*mu/rho/h/h)**2)**(-0.5)
            #combi_t =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u - (1-theta)*u0))) + grad((theta*p - (1-theta)*p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh)  
            combi_t_l =  combi_f * supg_t_ * inner((rho * dot(u0, nabla_grad((theta*u))) + grad((theta*p))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
            combi_t_r =  combi_f * supg_t_ * inner((rho * dot(u0, (1-theta)*nabla_grad((u0))) + (1-theta)*grad((p0))),(rho * dot(u0, nabla_grad(v)) - grad(q))) * dx(domain = mesh) 
        else:
            combi_t_l = combi_t_r = 0
        
        
                                            
        #Form calculation 
        correction_t_r = temam_t_r + epsilon_t_r + conv_t_r + bcflow_t_r + supg_t_r + combi_t_r +bc_3
        correction_t_l = temam_t_l + epsilon_t_l + conv_t_l + bcflow_t_l + supg_t_l + combi_t_l
        
        
        a = inner(u,v) * dx + (theta) * (a2) + correction_t_l
        L =  (1 - theta) * (a1 - L1) - (theta) * L2 - inner(u0,v)*dx + correction_t_r
        U = Function(W)
        solve(a == L, U, bcs)
        u0, p0 = U.split()
        
        
        
        #file work
        u0.rename("v0", "velocity")
        p0.rename("p0", "pressure")
        u_file.write(u0, t)
        p_file.write(p0, t)
        
        
        
        #time step apply
        t += dt
    
    
    
    #file close
    u_file.close()
    p_file.close()
    if plot_f == 1:
        plt.plot(t_arr,vc_arr)

2.1.3. Assignment (0.5 point) Compute the energy balance of the fully discrete problem, and identify possible sources of instabilities.

$\rho \dot{\vec{u}}+\rho(\vec{u} \cdot \nabla) \vec{u}-\mu \Delta \vec{u}+\nabla p=\vec{f}$ in $\Omega$

$\nabla \cdot \vec{u}=0$ in $\Omega$

$\vec{u}=\vec{u_D}$ in $\partial \Omega$

$\vec{u}(0)=\vec{u_0}$ in $\Omega$

we multiply by $\vec{u}$ and integrate over $\Omega$



$\rho \int_{\Omega} \dot{\vec{u_{n}}} \cdot \vec{u_{n}}+\rho \int_{\Omega}(\vec{u_{n}} \cdot \nabla) \vec{u_{n}} \cdot \vec{u_{n}}-\int_{\Omega} \mu_{n} \Delta \vec{u_{n}} \cdot \vec{u_{n}}$
$+\int_{\Omega} \nabla p \cdot \vec{u_{n}}=\int_{\Omega} \vec{f} \cdot \vec{u_{n}}$

$\frac{d}{d t} \frac{1}{2} \int\|\vec{u_n} \|^{2}+ \frac{\rho}{2} \int_{\Omega} \vec{u_n} \nabla\|\vec{u_n}\|^{2}+\mu \int_{\Omega}\|\nabla \vec{u_n}\|^{2}-\int_{\Omega} p \nabla \cdot \vec{u_n}=\int_{\Omega} \vec{f} \cdot \vec{u_n}+\int_{\Omega}\left(\mu \frac{\partial \vec{u_n}}{\partial n}-p \vec{n}\right) \cdot \vec{n}$

$\int_{\Omega} p \nabla \cdot \vec{u_n}=0$ for $\forall g_n \in Q_{n}$

$\int_{\Omega}\left(\mu \frac{\partial \vec{u_n}}{\partial n}-p \vec{n}\right) \cdot \vec{n} \ne 0$ from integration by parts of viscocity and pressure terms


so possible sources of instability are

$\frac{\rho}{2} \int_{\Omega} \vec{u_n} \nabla\|\vec{u_n}\|^{2}$

$\int_{\Omega} \vec{f} \cdot \vec{u_n}$

$\int_{\Omega}\left(\mu \frac{\partial \vec{u_n}}{\partial n}-p \vec{n}\right) \cdot \vec{n}$



2.1.4. Solve the problem using $\mathbb{P}_{2} / \mathbb{P}_{1}$ finite element spaces on the coarse mesh (stenosis_f0.6.xml) with $\theta=1, \tau=0.01$, and with $U_{\text {bulk }}$ such that the Reynolds number is
$$
R e=10 .
$$

In [7]:
folder_name = '6.2.1'

t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 0 
bcflow_f = 0
temam_f = 0
supg_f = 0
combi_f = 0


plot_f = 0

Re = 10

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f, folder_name)


% = 0 , time mplsd = 0
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may tak

2.1.5. Assignment (1 point) Solve the Navier-Stokes problem for
$$
R e=\{500,2000\}
$$
and plot all velocity fields together and simultaneously (i.e., for all Reynolds numbers) over time in Paraview. Do the simulations remain stable?

2.1.5 re = 500

In [9]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 0
temam_f = 0
supg_f = 0
combi_f = 0


plot_f = 0

Re = 500

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f ,folder_name)

% = 0 , time mplsd = 1
% = 10 , time mplsd = 40
% = 20 , time mplsd = 78
% = 30 , time mplsd = 117
% = 40 , time mplsd = 156
% = 50 , time mplsd = 195
% = 60 , time mplsd = 234
% = 70 , time mplsd = 274
% = 80 , time mplsd = 315
% = 90 , time mplsd = 356
% = 100 , time mplsd = 397


2.1.5 re = 2000

In [10]:
t_start = time.time()
T = 4.           # final time
num_steps = 400    # number of time steps
dt = T/num_steps
mu = 0.035            # kinematic viscosity
rho = 1.2            # density
epsilon = 0
theta = 1
R = 1
element_n = 2 #P? element to solve


progress_f = 1
proceents = 10


mesh_f= 0 # 0 for coarse 1 for fine
conv_f = 1 
bcflow_f = 0
temam_f = 0
supg_f = 0
combi_f = 0


plot_f = 0

Re = 2000

stokes_problem(T, num_steps, mu, rho, epsilon, theta, Re, R, element_n, plot_f, progress_f, proceents, t_start, mesh_f, conv_f, bcflow_f, temam_f, supg_f, combi_f, folder_name)

% = 0 , time mplsd = 1
% = 10 , time mplsd = 41
% = 20 , time mplsd = 83
% = 30 , time mplsd = 125
% = 40 , time mplsd = 167
% = 50 , time mplsd = 209
% = 60 , time mplsd = 250
% = 70 , time mplsd = 292
% = 80 , time mplsd = 335
% = 90 , time mplsd = 379
% = 100 , time mplsd = 421


2.1.6. 

Assignment (0.5 point) If they (or some) do not remain stable, what can be the theoretical explanation and for which simulation time do they become unstable for each of the Reynolds numbers? From the possible causes of the instabilities, which one is the most likely to be causing instabilities in your simulations?